# Footsies

This notebook implements different algorithms as player types for a simple adversarial turn-based game inspired by the fighting game Footsies.

The game is implemented in text-based form in the `game.py` file. Players are defined by an abstract class, and all contain an `act()` method that takes as input the current state of the game and outputs a Move.

In [ ]:
import math
import random
import numpy as np
from game import Move, State, Player, MoveSelection, Footsies

## Manual player

First, we define a player type that can be controlled through keyboard input. It also informs the human player of the state of the game in a more detailed way ; Later algorithms are provided the same information.

In [ ]:
class ManualPlayer(Player):
    name = ""
    def __init__(self, name: str = "Manuel"):
        self.name = name
    
    def act(self, game_state: State) -> Move:
        print(f"Your turn, {self.name}!{f' {game_state.turns_left} turns left.' if game_state.turns_left > -1 else ''}")
        print(f"You have {game_state.own_blocks} blocks left{' and have landed an attack' if game_state.own_has_attack else ''}.")
        print(f"Your opponent has {game_state.other_blocks} blocks left{' and has landed an attack' if game_state.other_has_attack else ''}.")
        
        move_str = ""
        while move_str not in MoveSelection:
            move_str = input("Choose your move: ")

        return MoveSelection[move_str]

## Random player

As a test, and as a way to measure the effectiveness of the different algorithms, we implement a class that selects a move at random.

In [ ]:
class RandomPlayer(Player):
    name = "The Chaotic"

    def act(self, game_state: State) -> Move:
        return random.choice(list(MoveSelection.values()))

## Simple counter strategy

Since some specific decisions in the game states are definite wins or losses, we can create a simple algorithm that avoids these bad decisions at all costs, at the risk of predictability.

In [ ]:
class CounterPlayer(Player):
    name = "The Simple-Minded"

    def act(self, game_state: State) -> Move:
        if game_state.other_has_attack:
            print("b")
            return MoveSelection["b"]  # Always block if the opponent has attack power
        
        if game_state.other_blocks == 0:
            print("a")
            return MoveSelection["a"]  # If they can't block, attack them
        
        if game_state.other_previous_move == None:
            return MoveSelection["b"] # Block to stay safe on the first turn

        if game_state.other_previous_move == MoveSelection["a"]:
            return MoveSelection["b"]  # If they attacked last, block
        if game_state.other_previous_move == MoveSelection["b"]:
            return MoveSelection["g"]  # If they blocked last, grab
        if game_state.other_previous_move == MoveSelection["g"]:
            return MoveSelection["a"]  # If they grabbed last, attack
    
        return MoveSelection["b"]

# Alternative counter strategy

Beyond reacting to the last move, we can introduce some randomness and risk factor based on a short form of history.

In [ ]:
import random

class BluffMaster(Player):
    name = "The High Roller"

    def __init__(self, bluff: float = 0.2):
        self.bluff_frequency = bluff
        self.enemy_tendency = 0 # becomes positive if agressive, negative if passive
    
    def act(self, game_state: State) -> Move:
        # Sometimes, pick a random move.
        if game_state.other_previous_move is not None:
            if game_state.other_previous_move != MoveSelection["b"]:
                self.enemy_tendency += 1
            else:
                self.enemy_tendency -= 1
        
        if random.random() < self.bluff_frequency:
            chosen_move = random.choice(list(MoveSelection.values()))
            return chosen_move

        if self.enemy_tendency > 1:
            return MoveSelection["b"]
        if self.enemy_tendency < 1:
            return MoveSelection["a"]
        
        # Otherwise, try to setup for a future move.
        print(f"{self.name} chooses to Grab to set up an attack.")
        return MoveSelection["g"]

# Markov Chain

We can approach this game from the perspective of Markov chains, where the states are the moves, and we can calculate compound probabilities for each sequence of moves from the opponent.

In [ ]:
import random
from collections import defaultdict

class MarkovChainPlayer(Player):
    name = "MarkovChain"
    
    def __init__(self, risk: float = 0.2):
        # Use a nested dictionary to track transition counts
        # For example: transitions[from_move][to_move] = count
        self.transitions = defaultdict(lambda: defaultdict(int))
        self.last_opponent_move = None
        self.risk = risk # introduce a risk factor to sometimes throw in dps in winning situations.
    
    def act(self, game_state: State) -> Move:
        fallback_choice = random.choice([MoveSelection["a"], MoveSelection["g"], MoveSelection["b"]])

        current_opponent_move = game_state.other_previous_move
        if self.last_opponent_move is not None:
            # Record the transition from the previous move to the current move
            self.transitions[self.last_opponent_move.name][current_opponent_move.name] += 1
        else:
            return fallback_choice
        
        # Predict the opponent's next move based on the current move patterns.
        predicted_move = None
        if self.transitions[current_opponent_move.name]:
            # Pick the move with the highest count following the current move.
            predicted_move = max(self.transitions[current_opponent_move.name], key=self.transitions[current_opponent_move.name].get)
        
        self.last_opponent_move = current_opponent_move

        if predicted_move == "Attack":
            if random.random() < self.risk:
                return MoveSelection["dp"]

            return MoveSelection["b"]
        elif predicted_move == "Block":
            return MoveSelection["g"]
        elif predicted_move == "Grab":
            if random.random() < self.risk:
                return MoveSelection["dp"]
            
            return MoveSelection["a"]
        elif predicted_move == "Dragon Punch":
            return MoveSelection["b"]

        return fallback_choice

# Testing

In [ ]:
player1 = BluffMaster()
player2 = MarkovChainPlayer()
game = Footsies(player1, player2)

player1success = 0
player2success = 0
for i in range(0, 1000):
    game = Footsies(player1, player2)
    result = game.start()
    if result == 1:
        player1success += 1
    else:
        player2success += 1

print(player1success, player2success)